In [1]:
# -*- coding: utf-8 -*-
from functools import reduce
from quant_utils.performance import Performance
import data_functions.portfolio_data as pds
import data_functions.fund_data as dfs
from quant_utils.constant import DB_CONN_JJTG_DATA
import quant_utils.data_moudle as dm
from quant_utils.utils import cut_series_to_group, cal_series_rank
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import dataframe_image as dfi
from joblib import Parallel, delayed
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号


def query_fund_alpha(trade_dt: str)->pd.DataFrame:
    query_sql = f"""
    SELECT
        END_DATE,
        TICKER_SYMBOL,
        `LEVEL`,
        INDICATOR,
        3M,
        6M,
        1Y,
        2Y,
        3Y
    FROM
        fund_derivatives_fund_alpha_performance 
    WHERE
        1 = 1 
        AND END_DATE = '{trade_dt}' 
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql).dropna()
    df = df.set_index(["END_DATE", "TICKER_SYMBOL", "LEVEL", "INDICATOR"])
    df = df.stack(dropna=False).reset_index().rename(columns={0: 'VALUE', "level_4": "PERIOD"})
    return df

def query_fund_type(trade_dt: str)->pd.DataFrame:
    query_sql = f"""
    SELECT
        TICKER_SYMBOL,
        LEVEL_1,
        LEVEL_2,
        LEVEL_3
    FROM
        fund_type_own_temp 
    WHERE
        1 = 1 
        AND REPORT_DATE = (
        SELECT
            max( REPORT_DATE ) 
        FROM
            fund_type_own_temp 
        WHERE
            PUBLISH_DATE <= '{trade_dt}'
    )
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df.set_index(["TICKER_SYMBOL"], inplace=True)
    df = df.stack().reset_index().rename(columns={"level_1": "LEVEL", 0: 'LEVEL_NAME'})
    return df

def query_fund_alpha_rank(trade_dt: str)->pd.DataFrame:
    alpha_perf = query_fund_type(trade_dt).merge(query_fund_alpha(trade_dt))
    alpha_perf["VALUE"] = alpha_perf.groupby(
        by=["LEVEL", "LEVEL_NAME", "INDICATOR", "PERIOD"]
    )["VALUE"].rank(pct=True, ascending=False, na_option='keep')

    alpha_perf.loc[alpha_perf["INDICATOR"].isin(["MAXDD", "ANNUAL_VOL"]), "VALUE"] = 1 - alpha_perf.loc[alpha_perf["INDICATOR"].isin(["MAXDD", "ANNUAL_VOL"]), "VALUE"]
    return alpha_perf

def query_fund_alpha_score(trade_dt: str)->pd.DataFrame:
    alpha_perf_rank = query_fund_alpha_rank(trade_dt)
    indicator_weight = {
        "CUM_ALPHA": 0.3,
        "IR": 0.3,
        "MAXDD": 0.2,
        "ANNUAL_VOL": 0.2,
    }
    period_weight = {
        "3M": 0.2,
        "6M": 0.2,
        '1Y': 0.2,
        '2Y': 0.2,
        '3Y': 0.2,
    }
    alpha_perf_rank['INDICATOR_WEIGHT'] = alpha_perf_rank['INDICATOR'].map(indicator_weight).fillna(0)
    alpha_perf_rank['PERIOD_WEIGHT'] = alpha_perf_rank['PERIOD'].map(period_weight).fillna(0)
    alpha_perf_rank["VALUE"] = (
        alpha_perf_rank["VALUE"].mul(
            alpha_perf_rank["INDICATOR_WEIGHT"], 
            axis=0
        ).mul(
            alpha_perf_rank["PERIOD_WEIGHT"], 
            axis=0
        )
    )
    alpha_perf_score = (
        alpha_perf_rank
        .groupby(by=["TICKER_SYMBOL", "LEVEL", "LEVEL_NAME"])["VALUE"]
        .sum()
        .reset_index()
    )
    alpha_perf_score["VALUE"] = alpha_perf_score["VALUE"].replace(0, np.nan)
    return alpha_perf_score

def query_fund_alpha_score_rank(trade_dt: str)->pd.DataFrame:
    alpha_perf_score = query_fund_alpha_score(trade_dt)
    alpha_perf_score['RANK'] = (
        alpha_perf_score
        .groupby(by=["LEVEL", "LEVEL_NAME"])["VALUE"]
        .rank(pct=True, na_option='keep', ascending=False)*100
    )
    return alpha_perf_score

In [5]:
trade_dt = "20240510"
file_path = 'E:/excel/渣打.xlsx'
df = pd.read_excel(file_path, sheet_name="代销")
df["TICKER_SYMBOL"] = df["TICKER_SYMBOL"].astype("str")
df["TICKER_SYMBOL"] = df["TICKER_SYMBOL"].str.rjust(6,"0")
df = df.merge(dfs.query_fund_info(), how='left')
df = df.merge(query_fund_alpha_score_rank(trade_dt), how='left')
df.to_excel("e:/excel/渣打筛选.xlsx", index=False)